This code was used for thin airfoil theory calculations

Setting up libraries and constants

In [ ]:
import numpy as np
import scipy.integrate as integrate
import math
import pandas as pd
pi = math.pi
radian = pi/180
degree = "\u00B0"
alpha = "\u03B1"
rho = 1.225 #kg/m^3
mu = 1.789*10**-5


In [ ]:
def to_subscript(n):
    subscript_map = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    return str(n).translate(subscript_map)

Defining the zero-lift angle of attack

In [ ]:
def alpha_zero_lift(airfoil,c):
    if airfoil[0] == "0" and airfoil[1] == "0":
        alpha_0_l = 0
        return alpha_0_l
    else:
        P = 0.1*float(airfoil[1])
        M = 0.01*float(airfoil[0])
        theta_zero_values = []
        x = np.linspace(0,c,100)
        theta_zero_P = math.acos(1-2*P)*radian
        for i in range(len(x)):
            theta_zero = math.acos(1-2*(x[i]/c)) * radian
            theta_zero_values.append(theta_zero)


Calculation alpha_zero lift

In [ ]:
# Integrand for the first integral
        integrand_1 = lambda theta_zero: (M / (P ** 2)) * (P - 0.5 * (1 - math.cos(theta_zero)) * (math.cos(theta_zero) - 1))
        result_1, _ = integrate.quad(integrand_1, 0, theta_zero_P)

        # Integrand for the second integral
        integrand_2 = lambda theta_zero: (M / (1 - (P ** 2))) * (P - 0.5 * (1 - math.cos(theta_zero)) * (math.cos(theta_zero) - 1))
        result_2, _ = integrate.quad(integrand_2, theta_zero_P, pi)

        # Calculating alpha_0_l
        alpha_0_l = -1/pi * (result_1 + result_2)
        return alpha_0_l

Calculation the Fourier coefficients

In [ ]:
def Fourier_coeff(airfoil,c,n,alpha):
    P = 0.1*float(airfoil[1])
    M = 0.01*float(airfoil[0])
    theta_zero_values = []
    x = np.linspace(0,c,100)
    theta_zero_P = math.acos(1-2*P)*radian
    for i in range(len(x)):
        theta_zero = math.acos(1-2*(x[i]/c)) * radian
        theta_zero_values.append(theta_zero)
    if M == 0 and P == 0:
        if n == 0:
            A_n = alpha
            return A_n
        else:
            A_n = 0
            return A_n
    else:
        if n == 0:
            integrand_1 = lambda theta_zero: alpha * radian - (M / (P ** 2)) * (P - 0.5 * (1 - math.cos(theta_zero)) * (math.cos(theta_zero) - 1))
            result_1, _ = integrate.quad(integrand_1, 0, theta_zero_P)

            # Integrand for the second integral
            integrand_2 = lambda theta_zero: (M / ((1 - P) ** 2)) * (P - 0.5 * (1 - math.cos(theta_zero)) * (math.cos(theta_zero) - 1))
            result_2, _ = integrate.quad(integrand_2, theta_zero_P, pi)

        else:
            integrand_1 = lambda theta_zero: (M / (P ** 2)) * (P - 0.5 * (1 - math.cos(theta_zero)) * (math.cos(theta_zero * n)))
            result_1, _ = integrate.quad(integrand_1, 0, theta_zero_P)

            integrand_2 = lambda theta_zero: (M / (1 - (P ** 2))) * (P - 0.5 * (1 - math.cos(theta_zero)) * (math.cos(theta_zero * n)))
            result_2, _ = integrate.quad(integrand_2, theta_zero_P, pi)

            # Calculating A_n
            A_n = 2/pi * (result_1 + result_2)
            return A_n

Calculating Cl and Cm for the three airfoils

In [ ]:
angles_of_attack = np.linspace(-4,24,29)
Cl_array_0012 = np.array([2 * pi * (angle * radian-alpha_zero_lift("0012", 1) ) for angle in angles_of_attack])
Cm_array_0012 = np.array([-((cl/4)+(pi/4) * (Fourier_coeff("0012", 1, 2,alpha=0) - Fourier_coeff("0012", 1, 1, alpha=0))) for cl in Cl_array_0012])

Cl_array_2412 = np.array([2 * pi * (angle * radian-alpha_zero_lift("2412", 1) ) for angle in angles_of_attack])
Cm_array_2412 = np.array([-((cl/4)+(pi/4) * (Fourier_coeff("2412", 1, 2,alpha=0) - Fourier_coeff("2412", 1, 1, alpha=0))) for cl in Cl_array_2412])

Cl_array_2406 = np.array([2 * pi * (angle * radian-alpha_zero_lift("2406", 1) ) for angle in angles_of_attack])
Cm_array_2406 = np.array([-((cl/4)+(pi/4) * (Fourier_coeff("2406", 1, 2,alpha=0) - Fourier_coeff("2406", 1, 1, alpha=0))) for cl in Cl_array_2406])

Organizing the data in data frames

In [ ]:
NACA0012 = pd.DataFrame({
    f"{alpha}\u2080\u2097": [alpha_zero_lift("0012", 1) / radian] * len(angles_of_attack),
    f"{alpha}":angles_of_attack,
    f"C{to_subscript('l')}": Cl_array_0012,
    f"C{to_subscript('m')}": Cm_array_0012
})

NACA2412 = pd.DataFrame({
    f"{alpha}\u2080\u2097": [alpha_zero_lift("2412", 1) / radian] * len(angles_of_attack),
    f"{alpha}":angles_of_attack,
    f"C{to_subscript('l')}": Cl_array_2412,
    f"C{to_subscript('m')}": Cm_array_2412
})

NACA2406 = pd.DataFrame({
    f"{alpha}\u2080\u2097": [alpha_zero_lift("2406", 1) / radian] * len(angles_of_attack),
    f"{alpha}":angles_of_attack,
    f"C{to_subscript('l')}": Cl_array_2406,
    f"C{to_subscript('m')}": Cm_array_2406
})
print(f"NACA2406\n{NACA2406}\n\nNACA2412\n{NACA2412}\n\nNACA0012\n{NACA0012}")
NACA0012.to_csv("NACA 0012 - Thin airfoil analysis.csv", sep=',')
NACA2412.to_csv("NACA 2412 - Thin airfoil analysis.csv", sep=',')
NACA2406.to_csv("NACA 2406- Thin airfoil analysis.csv", sep=',')